## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/pytorch/fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 28172, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (339/339), done.
remote: Total 28172 (delta 308), reused 465 (delta 234), pack-reused 27586
Receiving objects: 100% (28172/28172), 11.87 MiB | 21.63 MiB/s, done.
Resolving deltas: 100% (21127/21127), done.


In [4]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 11.25 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [5]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [6]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [7]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [8]:
%cd /tmp/fairseq/

/tmp/fairseq


In [9]:
!git checkout a8fe9434adeb6caea65b34ca968f4e5cd1d75be6

Note: checking out 'a8fe9434adeb6caea65b34ca968f4e5cd1d75be6'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a8fe9434 fix readme link (#1890)


In [10]:
!pip install .

Processing /tmp/fairseq
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - done
    Preparing wheel metadata ... - \ done
     |████████████████████████████████| 54 kB 1.1 MB/s 
     |████████████████████████████████| 123 kB 4.0 MB/s 
     |████████████████████████████████| 112 kB 4.5 MB/s 
  Created wheel for fairseq: filename=fairseq-1.0.0a0+a8fe943-cp37-cp37m-linux_x86_64.whl size=1994409 sha256=6c71cbb451f0c840872dad220296d13f441b1175d3f052255376ef1a7199f224
  Stored in directory: /tmp/pip-ephem-wheel-cache-ogxbauhu/wheels/e3/52/72/c9070e4abac84e3448568d15e13e9e42bb689c08614271d8f3
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=71b170fa6ca6784919a607529d7505d3f4579b3ee3591c5a808b7b2106e7474e
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824bae

In [11]:
%cd /tmp/fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [12]:
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [13]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

CLUS128		    precompute_pca512_cls128_mean	  train.npy
__notebook__.ipynb  precompute_pca512_cls128_mean_pooled  train.phn
__output__.json     test.lengths			  train.tsv
__results__.html    test.ltr				  train.wrd
custom.css	    test.npy				  valid.lengths
dict.test	    test.phn				  valid.ltr
dict.train	    test.tsv				  valid.npy
dict.valid	    test.wrd				  valid.phn
pca		    train.lengths			  valid.tsv
precompute_pca512   train.ltr				  valid.wrd


In [14]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir config/gan \
	--config-path . \
	--config-name w2vu \
	task._name='unpaired_audio_text' \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [15]:
!bash rungan.sh

[2021-05-30 20:33:51,119][HYDRA] Launching 5 jobs locally
[2021-05-30 20:33:51,119][HYDRA] 	#0 : task._name=unpaired_audio_text task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=False checkpoint.save_dir=/kaggle/working common.seed=0
[2021-05-30 20:33:53,234][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tb', 'wandb_project': None, 'azureml_logging': False, 'seed': 0, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': True, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_l